In [1]:
# Cpython：内置类型的方法不会调用子类覆盖的方法
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [2]:
# 继承至dict的__init__方法，会忽略覆盖的__setitem__方法
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
# 继承至dict的update方法，也会忽略覆盖的__setitem__方法
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [5]:
# 菱形问题：由不相关的祖先类实现同名方法引发的命名冲突
class A:
    def ping(self):
        print('ping:', self)

class B(A):
    def pong(self):
        print('pong:', self)

class C(A):
    def pong(self):
        print('PONG:', self)

class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)


In [7]:
d = D()
d.pong()

pong: <__main__.D object at 0x00000221CE770970>


In [8]:
C.pong(d)

PONG: <__main__.D object at 0x00000221CE770970>


In [9]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [10]:
d.ping()

ping: <__main__.D object at 0x00000221CE770970>
post-ping: <__main__.D object at 0x00000221CE770970>


In [11]:
d.pingpong()

ping: <__main__.D object at 0x00000221CE770970>
post-ping: <__main__.D object at 0x00000221CE770970>
ping: <__main__.D object at 0x00000221CE770970>
pong: <__main__.D object at 0x00000221CE770970>
pong: <__main__.D object at 0x00000221CE770970>
PONG: <__main__.D object at 0x00000221CE770970>


In [13]:
import tkinter
tkinter.Text.__mro__

(tkinter.Text,
 tkinter.Widget,
 tkinter.BaseWidget,
 tkinter.Misc,
 tkinter.Pack,
 tkinter.Place,
 tkinter.Grid,
 tkinter.XView,
 tkinter.YView,
 object)

In [14]:
# 处理多重继承
# 01. 把接口继承和实现继承区分开
# 继承接口，创建子类型，实现是什么关系
# 继承实现，通过重用避免代码重复
# 02. 使用抽象基类显式表示接口
# 如果类的作用是定义接口，应该明确把他定义为抽象基类
# 03. 通过混入（mixin）重用代码
# 如果一个类的作用是为多个不相关的子类提供方法实现
# 从而实现重用，但不体现‘是什么’关系，应该把这个类明确定义为混入类（mixin class）
# 04. 在名称中明确指明混入
# 05. 抽象基类可以作为混入，反之则不成立
# 06. 不要子类化多个具体的类
# 07. 为用户提供聚合类
# 08. 优先使用对象组合，而不是类继承

In [1]:
# 可迭代对象
# Sentence类第一版：单词序列
# 传入文本，逐个单词迭代
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]

    def __len__(self):
        return len(self.words)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [2]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [4]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [5]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [6]:
s[0]

'The'

In [7]:
s[-1]

'said'

In [1]:
# 序列可以迭代的原因
# 解释器需要迭代对象x时，会自动调用iter(x)
# 内置的iter函数有一下作用：
# 1） 检查对象是否实现了__iter__方法，如果实现了就调用它，获取一个迭代器
# 2） 如果没有实现__iter__方法，但实现了__getitem__方法，
#     python会创建一个迭代器，尝试按顺序获取元素
# 3） 如果尝试失败，python会抛出TypeError异常

In [2]:
# 可迭代对象（iterable）
# 使用iter()可以获取迭代器的对象
# 序列都是可以迭代的
s = 'ABC'
for char in s:
    print(char)

A
B
C


In [4]:
# 如果没有for语句，可以使用while循环模拟
s = 'ABC'
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

A
B
C


In [3]:
s = Sentence('Pig and Pepper')

In [4]:
it = iter(s)
it

In [5]:
next(it)

'Pig'

In [6]:
next(it)

'and'

In [7]:
next(it)

'Pepper'

In [8]:
next(it)

StopIteration: 

In [9]:
list(it)

[]

In [10]:
list(s)

['Pig', 'and', 'Pepper']

In [1]:
# 迭代器
# 实现了无参数的__next__方法，返回序列中的下一个元素
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0

    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()

        self.index += 1
        return word

    def __iter__(self):
        return self

In [1]:
# Sentenc类第3版：生成器函数
# 只要Python函数的定义体中有yield关键字，该函数就是生成器函数
# 调用生成器函数时，会返回一个生成器对象
# 也就是说，生成器函数时生成器工厂
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:
            yield word
        return

In [2]:
# Sentence类第4版：惰性实现
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group

In [1]:
# 等差数列生成器
# 典型的迭代器 - 遍历数据结构
class ArithmeticProgression:
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None -> 无穷数列

    def __iter__(self):
        result = type(self.begin + self.step)(self.begin) # 将self.begin赋值给result， 不过会强制转换为前面的加法算式得到的类型
        forever = self.end is None # True or False
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index

In [3]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [4]:
ap = ArithmeticProgression(0, .5, 3)
list(ap)

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5]

In [5]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [6]:
# 生成器函数
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [7]:
ap = aritprog_gen(0, 1, 3)
list(ap)

[0, 1, 2]

In [8]:
# 使用itertools模块生成等差数列
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

In [10]:
ap = aritprog_gen(0, 1, 3)
list(ap)

[0, 1, 2]

In [11]:
# 标准库中的生产器函数
# 用于过滤的生成器函数
def vowel(c):
    return c.lower() in 'aeiou'

# filter(predicate, it)
# 把it中的各个元素传给predicate，如果predicate(item)
# 返回真值，那么产出对应元素；如果predicate为None
# 那么只产出真值元素
list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [12]:
# itertools.filterfalse(predicate, it)
# 与filter作用相反
import itertools
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [13]:
# itertools.dropwhile(predicate, it)
# 处理it，跳过predicate的计算结果为真的元素（遇到第一个假值为止）
# 然后产出剩下的各个元素（不再进一步检查）
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [14]:
# itertools.takewhile(predicate, it)
# 处理it，返回predicate的计算结果为真的元素(遇假则停)
# 然后立即停止（不再进一步检查）
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [15]:
# itertools.compress(it, selector_it)
# 并行处理2个可迭代对象；如果selector_it中的元素为真值
# 产出it中对应的元素
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [16]:
# itertools.islice(it, stop) or itertools.islice(it, start, stop, step=1)
# 产出it的切片，作用类似s[:stop] 或者s[start:stop:step]
# 不过it可以是任何可迭代的对象，且这个函数实现的是惰性操作
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [17]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [18]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

In [19]:
# 用于映射的生成器函数

# itertools.accumulate(it, [func])
# 产出累计的总和；如果提供了func，那么
# 将前2个元素传给他，然后把计算结果
# 和下一个元素传给他，以此类推，最后产出结果
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

import itertools

list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [20]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [21]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [22]:
# enumerate(iterable, start=0)
# 产出由两个元素组成的元组，结构是(index, item)
# 其中index从start开始计数，item则从iterable中获取
list(enumerate('albatroz', 1))

[(1, 'a'),
 (2, 'l'),
 (3, 'b'),
 (4, 'a'),
 (5, 't'),
 (6, 'r'),
 (7, 'o'),
 (8, 'z')]

In [23]:
# map(func, it1, [it2, ..., itN])
# 把it中的各个元素传给func，产出结果
# 如果传入N个可迭代对象，那么func必须
# 能接受N个参数，而且要并行处理各个可迭代对象
import operator

list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [24]:
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [25]:
list(map(lambda a,b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [28]:
# itertools.starmap(func, it)
# 把it中的各个元素传给func，产出结果
# 输入的可迭代对象应该可以产出可迭代元素iit
# 然后以func(*iit)这种形式调用func
list(itertools.starmap(lambda a, b: b / a, enumerate(itertools.accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

In [29]:
# 合并多个可迭代对象的生成器函数

# itertools.chain(it1, it2, ..., itN)
# 先产出it1中的所有元素，然后产出it2中的所有元素
# 以此类推，无缝连接在一起
import itertools

list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [30]:
# itertools.chain.from_iterable(it)
# 产出it生成的各个可迭代对象中的元素，
# 一个接一个，无缝连接在一起
# it应该产出可迭代的元素，例如可迭代对象列表

list(itertools.chain(enumerate('ABC'))) # itertools.chain传入一个可迭代对象的时候不起作用

[(0, 'A'), (1, 'B'), (2, 'C')]

In [31]:
# 但是itertools.chain.from_iterable函数从迭代对象中获取
# 每一个元素，然后按顺序把元素连接起来，前提是这些元素本身也是可迭代的对象
list(itertools.chain.from_iterable(enumerate('ABC'))) 

[0, 'A', 1, 'B', 2, 'C']

In [32]:
# zip(it1, it2, ..., itN)
# 并行从输入的各个可迭代对象中获取元素
# 产出由N个元素组成的元组，只要一个可迭代对象到头了就停止
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [33]:
# itertools.zip_longest(it1, it2, ..., itN, fillvalue=None)
# 等到最长可迭代对象到头才停止，空缺值使用fillvalue填充
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]